# Livrable 4 : Groupe 08


# ATTENTION !
## Ce programme supprime tous les fichiers .csv & .wav dans le dossier où il est situé !



## Importation des bibliothèques

In [1]:
import tkinter as tk
from tkinter import *
from tkinter import ttk #Nouveau widget tkinter ajoutés en 2007
from tkinter import filedialog
import time
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
from scipy.io import wavfile
import pyaudio
import scipy.signal
import pandas as pd
import os
import csv

## Code principal

In [ ]:
#On définit la classe qui va s'occuper de toute la partie démodulation.

class Demodulation:
    """
    Traitement inverse (Démodulation du signal)
    """

    
    def __init__(self, source):              
        CSV = np.loadtxt(source)
        Demodulation.demod(CSV, "ASK", "LRC")
        Demodulation.demod(CSV, "FSK", "LRC")
        
    def demod(data, modType, CheckType):  #Fonction qui va s'occuper de démoduler le message de deux manières différentes:
        if modType == "ASK":              # ASK et FSK
            Ap = 1
            Fe = 44100   #Fréquence d'échantillonnage
            Fp = 2000    #Fréquence de la proteuse ASK
            baud = 30    #Débit en bit/s
            Ns = Fe/baud  #Nombre de symbole dans le message
            Nbits = len(data)/Ns  #Nombre de bit dans le message
            N=Nbits*Ns 
            
            t_ASK= np.arange(0, N/Fe, 1/Fe)  #Vecteur temps
            S = Ap*np.sin(2*np.pi*Fp*t_ASK)  #Porteuse ASK
            
            Produit= S*data
            
            Res= []                                
            N, Ns = int(N), int(Ns)
            i=0
            for i in range(0,N,Ns):      #Aire de la courbe pour chaque bit
                Res.append (np.trapz(Produit[i:i+Ns],t_ASK[i:i+Ns]))
                
            message_demodule = np.array(Res) > 0    #Renvoie true si l'aire est positif, false sinon
            
            message_demodule_ASK= []
            
            for ii in range (0,len(Res)):         #On remplace True et False par 1 et -1
    
                if message_demodule [ii] == True:
                    message_demodule_ASK.extend([int(1)]) 
                else:
                    message_demodule_ASK.extend([int(-1)]) 
                    
            Demodulation_ASK =  [0 if i==-1 else 1 for i in message_demodule_ASK] #On remplace 1 et -1 par 1 et 0

            message_recu_bin = []  

            for i in range(0, len(Demodulation_ASK),2):           #On décode le Manchester.
                if Demodulation_ASK[i] == 1 and Demodulation_ASK[i+1] == 0:
                    message_recu_bin.append(0)
                elif Demodulation_ASK[i] == 0 and Demodulation_ASK[i+1] == 1:
                    message_recu_bin.append(1)
            
            message_recu_bin, LRC_recu = Demodulation.decode_checksum(message_recu_bin, CheckType)   #On isole les
                                                                                                    #bits de parités
            #print(message_recu_bin)                                                                 #ou le CRC
            
            bin_data = ""  #On tranforme la liste en string
            for element in message_recu_bin:
                bin_data += str(element)
                    
                
            data_reçu =' '
            
            CRC_temp = []      


            for i in range(0, len(bin_data), 7):          #On reconvertie chaque caractère qui était en bit en ASCII

                temp_data = int(bin_data[i:i+7])

                CRC_temp.append(str(temp_data))  

                decimal_data = Demodulation.BinaryToDecimal(temp_data)

                data_reçu = data_reçu + chr(decimal_data)

            #print(data_reçu)

            print("Le message reçu est:", data_reçu)   

            Erreur = Demodulation.CalculErreur(CRC_temp ,CheckType, LRC_recu)       #On détecte s'il y a une erreur de
            ASK_decode.configure(text="Le message recu en ASK est : " + data_reçu)  #transmission
            Error_ASK.configure(text=Erreur)           #On affiche le texte erreur
                
        elif modType == "FSK":                                       #On fait pareil pour le FSK
            #-------------[Définition des variables]-----------

            Fe = 44100
            baud = 30
            Ns = Fe/baud
            Nbits=len(data)/Ns
            N=Ns*Nbits
            fp1 = 500
            fp2=2000
            Fp = 2000
            A1= 1
            A2 = 1
            t_FSK = np.arange(0, N/Fe ,1/Fe)
            P1 = A1*np.sin(2*np.pi*fp1*t_FSK)
            P2 = A2*np.sin(2*np.pi*fp2*t_FSK)

            #------------[Démodulation]-------------------------

            s1 = P1*data
            s2 = P2*data

            Res= []     
            Res2=[]
            N, Ns = int(N), int(Ns)
            #print(N, Ns)

            for i in range(0,N,Ns):
                Res.append (np.trapz(s1[i:i+Ns],t_FSK[i:i+Ns]))
                Res2.append (np.trapz(s2[i:i+Ns],t_FSK[i:i+Ns]))


            Demodulation_FSK = []

            for k in range(len(Res)):
                if Res[k] > Res2[k]:
                    Demodulation_FSK.append(1)
                else:
                    Demodulation_FSK.append(0)

            message_recu_bin = []  

            for i in range(0, len(Demodulation_FSK),2):
                if Demodulation_FSK[i] == 1 and Demodulation_FSK[i+1] == 0:
                    message_recu_bin.append(0)
                elif Demodulation_FSK[i] == 0 and Demodulation_FSK[i+1] == 1:
                    message_recu_bin.append(1)

            
            message_recu_bin, LRC_recu = Demodulation.decode_checksum(message_recu_bin, CheckType)

            #print(LRC_recu)
            #print(message_recu_bin)

            bin_data = ""
            for elem in message_recu_bin:  
                bin_data += str(elem)

            data_reçu =' '
            
            CRC_temp = []


            for i in range(0, len(bin_data), 7): 

                temp_data = int(bin_data[i:i+7])

                CRC_temp.append(str(temp_data))  

                decimal_data = Demodulation.BinaryToDecimal(temp_data)

                data_reçu = data_reçu + chr(decimal_data)

            #print(data_reçu)

            print("Le message reçu est:", data_reçu) 

            Erreur = Demodulation.CalculErreur(CRC_temp ,CheckType, LRC_recu)
            FSK_decode.configure(text="Le message recu en FSK est : " + data_reçu)
            Error_FSK.configure(text=Erreur)
            

    def CalculErreur(bin_tab, checktype, CheckRecu):         #Fonction qui s'occupe de voir s'il y a une erreur dans
        if checktype == "LRC":                               #la transmission.
            LRC_calcule = Traitement.CheckSum(bin_tab ,checktype)

            if LRC_calcule == CheckRecu:
                print("Il ne semble pas y avoir d'erreur.")
                return "Il ne semble pas y avoir d'erreur. (d'après les bits de parités)"
            else:
                print("Il semble y avoir une erreur dans le message.")
                return "Il  semble y avoir des erreurs. (d'après les bits de parités)"
                
            
        
        elif checktype == "VRC":
            print("no")
            
        elif checktype == "CRC":
            divisor = "1010"
            data =""
            for elem in CheckRecu:
                data += str(elem)
            reminder = int(Traitement.mod2div(data, divisor))
            if reminder == 0:
                print("Il ne semble pas y avoir d'erreur.")
                return "Il ne semble pas y avoir d'erreur. (d'après le CRC)"
            else:
                print("Il semble y avoir une erreur dans le message.")
                return "Il  semble y avoir des erreurs. (d'après le CRC)"
        
    def BinaryToDecimal(binary):           #Fonction qui s'occupe de convertir un décimal en binaire
        decimal, i, n = 0, 0, 0

        while(binary != 0): 
            dec = binary % 10
            decimal = decimal + dec * pow(2, i) 
            binary = binary//10
            i += 1
        return (decimal)
    
    def decode_checksum(bin_tab, checktype):          #Cette fonction se charge de d'extraire les vérifications du
        if checktype == "LRC":                        #message
            LRC_recu=[]
            message_recu_bin = bin_tab
            for i in range(7,len(message_recu_bin),7):
                    LRC_recu.append(message_recu_bin.pop(i))
                    message_recu_bin.append(2)
                    
            index = 0
            for i in range(0,len(message_recu_bin)):
                 if message_recu_bin[i] == 2:
                    index = i
                    break
                        
            del message_recu_bin[index:len(message_recu_bin)]
                
            return message_recu_bin, LRC_recu
        
        if checktype == "VRC":
            print("no")
            
        else:
            message_binaire = bin_tab[0:(len(bin_tab)-3)]
            print(message_binaire)
            print(bin_tab)
            return message_binaire, bin_tab


#On définie la classe qui va se charger du traitement (modulation)
        
class Traitement:
    """
    Traitement du message (modulation)
    """
    def __init__(self, message, modulation, checksum):
        self.modulation = modulation
        self.checksumMode = checksum
        self.message = message
        self.ASCII = Traitement.DécoupeMessageASCII(self.message)
        self.Binaire = Traitement.ASCIIToBinary(self.ASCII)
        self.CheckSum = Traitement.CheckSum(self.Binaire, self.checksumMode)
        self.stringToInt = Traitement.stringListToInt(self.Binaire)
        self.MixLits = Traitement.MixLists(self.Binaire, self.CheckSum, self.checksumMode)
        self.Manchester = Traitement.ToManchester(self.MixLits)
        self.traitement = Traitement.modulation(self.Manchester, self.modulation, self.message)
        Traitement.csvWriter(self.traitement, self.message)
    
    def Tramming(nb_tram, data):            #Fonction qui devrait s'occuper des découpages en trames mais qui n'est pas
        flag = "0111110"                    #pas fonctionelle
        Protocole = "10101010" 
        adresse_list = Traitement.ASCIIToBinary(Traitement.DécoupeMessageASCII(str(len(Message.Mess))))
        adresse = str(adresse_list[0])
        Trams = []
        for i in range (1,nb_tram):
            print(i)
            controle_list = Traitement.ASCIIToBinary(Traitement.DécoupeMessageASCII(str(i)))
            controle = str(controle_list[0])
            Trams.append(flag + Protocole + controle + str(data)+flag)
            
        #print(Trams)
            
        
        
    def csvWriter(decimal_tab, nom):   #Fonction qui écrit les fichiers .csv
        df = pd.DataFrame(decimal_tab)
        df.to_csv(str(nom)+"_dec.csv", index=False, header=False)
        
    def modulation(binaire, modulation, nom):    #Fonction que se charge de la modulation, aussi bien ASK que FSK
        A1 =  1    #Amplitude de la porteuse 1                             
        A2 = 1   #Amplitude de la porteuse 2
        fp1 = 500   #Fréquence de la porteuse 1 
        fp2 = 2000   #Fréquence de la porteuse 2
        Fe = 44100                                              
        baud = 30    
        Ns = Fe/baud
        Nbits = len(binaire)                                                  
        N = Nbits*Ns 
        Extended_mix = np.repeat(binaire, Ns)
        if modulation == "FSK":                 
            t = np.arange(0, N/Fe ,1/Fe)
            P1 = A1*np.sin(2*np.pi*fp1*t)
            P2 = A2*np.sin(2*np.pi*fp2*t)
            print("Modulation type FSK en cours: \nFréquence porteuses : {} & {}\nBaud, Fe : {}, {} \nModulation terminée.\n".format(fp1, fp2, baud, Fe))
            
            FSK = []     

            for i in range (len(Extended_mix)):  
                if Extended_mix[i] == 1:
                    FSK.append(P1[i])
                else:
                    FSK.append(P2[i])

            #plt.plot(t[0:4000],FSK[0:4000])
            #plt.show()
            FSK_copy = np.array(FSK)*32767
            wavfile.write(str(nom)+"FSK.wav", Fe, FSK_copy.astype(np.int16)) 
            LireWAV(nom+"FSK.wav", Fe)
            
            f = Figure(figsize=(5,2), dpi=80)
            a = f.add_subplot(111)
            
            a.plot(t[0:4000],FSK[0:4000])
            
            canva = FigureCanvasTkAgg(f, master=root)  #On affiche le graphique dans la fenêtre d'envoie
            canva.draw()
            canva.get_tk_widget().grid(row=1, column = 0, columnspan=2, rowspan=1, stick=tk.S, pady=20)
            a.set_ylabel("Amplitude")
            a.set_xlabel("Temps (s)")
            a.set_title("Modulation FSK en codage Manchester")
            return FSK
                
        else:           
            Ap =  1                                 
            Fp = 2000             
            Fe = 44100                                              
            baud = 30    
            Ns = Fe/baud
            Nbits = len(binaire)                                                  
            N = Nbits*Ns 
            t= np.arange(0, N/Fe, 1/Fe)
            Porteuse = Ap*np.sin(2*np.pi*Fp*t)                  
            ASK = Extended_mix*Porteuse       
            print("Modulation type ASK en cours: \nFréquence porteuse : {} \nBaud, Fe : {}, {} \nModulation terminée.\n".format(Fp, baud, Fe))

            #plt.plot(t[0:4000], ASK[0:4000])
            #plt.show()
            ASK_copy = np.array(ASK)*32767
            wavfile.write(str(nom)+"ASK.wav", Fe, ASK_copy.astype(np.int16))  
            LireWAV(nom+"ASK.wav", Fe)
            
            f = Figure(figsize=(5,2), dpi=80)
            a = f.add_subplot(111)
            
            a.plot(t[0:4000],ASK[0:4000])
            
            canva = FigureCanvasTkAgg(f, master=root)
            canva.draw()
            canva.get_tk_widget().grid(row=1, column = 0, columnspan=2, rowspan=1, stick=tk.S, pady=20)
            a.set_ylabel("Amplitude")
            a.set_xlabel("Temps (s)")
            a.set_title("Modulation ASK en codage Manchester")
            
            return ASK
    
    def DécoupeMessageASCII(message):   #Fonction qui fait correspondre chaque caractère du message en son codage ASCII
        #print("\n")
        #print("Traitement du message {}".format(message))
        Message_Decoupe_ASCII = []
        for lettre in message:
            Message_Decoupe_ASCII.append(ord(lettre))
        #print("Découpage du message en liste de carcatère, ensuite converti en ASCII :")
        #print(Message_Decoupe_ASCII)
        #print("\n")
        return Message_Decoupe_ASCII

    def ASCIIToBinary(tab_ASCII):   #On convertie les nombres ASCII en binaire
        Message_Decoupe_bin = []
        for element in tab_ASCII:
            Message_Decoupe_bin.append(bin(element).replace("0b", ""))
        #print("Conersion ASCII → Binaire :")
        #print(Message_Decoupe_bin)
        #print("\n")
        return Message_Decoupe_bin
    
    def CheckSum(tab_binaire, mode):   #On calcul soit le LRC soit le VRC soit le CRC
        if mode == "LRC":
            LRC_list = []
            for element in tab_binaire:
                total = 0
                for i in range(len(element)):
                    total = total + int(element[i])
                LRC_list.append(total%2)
            #print("Calcule du bit de parité de chaque caractère :")
            #print(LRC_list)
            #print("\n")   
            return LRC_list

        elif mode == "VRC":
            print("Pas encore implémenté")
        else:
            divisor = "1010"
            bin_data =""
            for elem in tab_binaire:
                bin_data += str(elem)
            EncodedData = Traitement.encodeData(bin_data, divisor)
            Encoded_data_list =[]
            for i in range(0, len(EncodedData), 7):
                Encoded_data_list.append(str(EncodedData[i:i+7]))
            return Encoded_data_list
            
    def xor(a, b):
 
    # initialize result
        result = []

        # Traverse all bits, if bits are
        # same, then XOR is 0, else 1
        for i in range(1, len(b)):
            if a[i] == b[i]:
                result.append('0')
            else:
                result.append('1')

        return ''.join(result)
 
 
    # Performs Modulo-2 division
    def mod2div(divident, divisor):

        # Number of bits to be XORed at a time.
        pick = len(divisor)

        # Slicing the divident to appropriate
        # length for particular step
        tmp = divident[0 : pick]

        while pick < len(divident):

            if tmp[0] == '1':

                # replace the divident by the result
                # of XOR and pull 1 bit down
                tmp = Traitement.xor(divisor, tmp) + divident[pick]

            else: # If leftmost bit is '0'

                # If the leftmost bit of the dividend (or the
                # part used in each step) is 0, the step cannot
                # use the regular divisor; we need to use an
                # all-0s divisor.
                tmp = Traitement.xor('0'*pick, tmp) + divident[pick]

            # increment pick to move further
            pick += 1

        # For the last n bits, we have to carry it out
        # normally as increased value of pick will cause
        # Index Out of Bounds.
        if tmp[0] == '1':
            tmp = Traitement.xor(divisor, tmp)
        else:
            tmp = Traitement.xor('0'*pick, tmp)

        checkword = tmp
        return checkword
 
    # Function used at the sender side to encode
    # data by appending remainder of modular division
    # at the end of data.
    def encodeData(data, key):

        l_key = len(key)

        # Appends n-1 zeroes at end of data
        appended_data = data + '0'*(l_key-1)
        remainder = Traitement.mod2div(appended_data, key)

        # Append remainder in the original data
        codeword = data + remainder
        return codeword  

            
    def stringListToInt(list): #Fonction qui transforme un tableau de string en tableau de int.
        IntBinList = []
        for elem in list:
            IntBinList.append(int(elem))
        return IntBinList
        
    def MixLists(BinList, CRC, mode): #Place le bit de parité correspondant à chaque mot binaire.
        MixList = []
        if mode == "LRC":
            for i in range(len(BinList)):
                BinLetter = BinList[i]
                for y in range(len(BinLetter)):
                    MixList.append(int(BinLetter[y]))
                MixList.append(int(CRC[i]))
            #print("On insert le bit de parité de chaque caractère au bon endroit :")
            #print(MixList)
            #print("\n")
            return MixList

        elif mode == "VRC":
            print("non")
        else:
            for i in range(len(BinList)):
                BinLetter = BinList[i]
                for y in range(len(BinLetter)):
                    MixList.append(int(BinLetter[y]))
            return MixList

    def ToManchester(BinList): #Fonction qui code un message bianire en utilisant Manchester.
        Manchester_list = []
        for element in BinList:
            if element == 0:
                Manchester_list.append(1)
                Manchester_list.append(0)
            else:
                Manchester_list.append(0)
                Manchester_list.append(1)
        #print("On convvertit en Manchester :")
        #print(Manchester_list)
        #print("\n")
        return Manchester_list

            
    
class Message(Traitement): #classe qui va s'occuper de la création de chaque message
    """
    Permet de créer un nouveau message à transmettre avec comme paramètres : 
        -le message 
        -le type de modulation
        -le type de Checksum
    """
    
    Mess = []  #On stocke tous les messages créées dans cette liste
    NombreMessage = 0  #Total de message transmis
    
    def __init__(self, message, TypeModulation, TypeChecksum):
        self.id = Message.NombreMessage  
        self.modulation = TypeModulation
        self.checksum = TypeChecksum
        self.message = message.replace(" ", "_")
        Message.NombreMessage += 1
        super().__init__(self.message, self.modulation, self.checksum) #On transmet le message à la classe Traitement
        print("Création du {} message contenant {}{}{}".format(Message.NombreMessage, "'", message, "'"))
        

def RecupParam(ASK, FSK, VRC, LRC, CRC):  #Fonction qui sert à récupérer tous les paramètres d'encodement du message.
    modulation = "None"
    checksum = "None"
    if ASK == "1":
        modulation = "ASK"
    else:
        modulation = "FSK"  
    if VRC == "1":
        checksum ="VRC"  
    elif LRC == "1":
        checksum = "LRC" 
    else:
        checksum = "CRC"
        
    return modulation, checksum
        
        
def ButtonClicked(boutton):  #Fonction qui va s'occuper des actions de chaque boutons.
    if boutton == "valider":
        if (checkbox_ASK.get() == "0" and checkbox_FSK.get() == "0") or (checkbox_ASK.get() == "1" and checkbox_FSK.get() == "1"):
            ErrorModulation.grid(row=1, pady=25, padx=5 , stick=tk.NW, columnspan=2)
            ErrorCheckSum.grid_forget()
            Error.grid_forget()
            msg_transmis.grid_forget()
            ResetCanva()
        elif (checkbox_VRC.get() == checkbox_LRC.get() == checkbox_CRC.get() == "0") or (checkbox_VRC.get() == checkbox_LRC.get() == checkbox_CRC.get() == "1") or (checkbox_VRC.get() == checkbox_LRC.get() == "1") or (checkbox_CRC.get() == checkbox_LRC.get() == "1") or (checkbox_CRC.get() == checkbox_VRC.get() == "1"): 
            ErrorCheckSum.grid(row=1, pady=25, padx=5 , stick=tk.NW, columnspan=2)
            Error.grid_forget()
            ErrorModulation.grid_forget()
            msg_transmis.grid_forget()
            ResetCanva()
        elif message.get() != "":
            modulation, checksum = RecupParam(checkbox_ASK.get(), checkbox_FSK.get(), checkbox_VRC.get(), checkbox_LRC.get(), checkbox_CRC.get())
            NewMessage = Message(message.get(), modulation, checksum)
            Message.Mess.append(NewMessage)
            Transmissions.set("Nombre de transmition : {}".format(Message.NombreMessage))
            message2.set("Le message {} a été transmis avec succès !  (Modulation : {}, Checksum : {})".format(message.get(),modulation, checksum))      #Messages[Message.NombreMessage-1].message (pour afficher le message avec des tirés du bas)
            Error.grid_forget()
            ErrorModulation.grid_forget()
            ErrorCheckSum.grid_forget()
            msg_transmis.grid(row=1, pady=25, padx=5 , stick=tk.NW, columnspan=2)
            message.set("")


        else:
            Error.grid(row=1, pady=25, padx=5 , stick=tk.NW)
            msg_transmis.grid_forget()
            ErrorModulation.grid_forget()
            ErrorCheckSum.grid_forget()
            ResetCanva()
            
    elif boutton == "prédéfini":
        modulation = "FSK"
        checksum = "LRC"
        NewMessage = Message("SOS", modulation, checksum)
        Message.Mess.append(NewMessage)
        Transmissions.set("Nombre de transmition : {}".format(Message.NombreMessage))
        message2.set("Le message {} a été transmis avec succès ! (Modulation : {}, Checksum : {})".format("SOS",modulation, checksum))       #Messages[Message.NombreMessage-1].message (pour afficher le message avec des tirés du bas)
        Error.grid_forget()
        ErrorModulation.grid_forget()
        ErrorCheckSum.grid_forget()
        msg_transmis.grid(row=1, pady=25, padx=5 , stick=tk.NW, columnspan=2)
        message.set("")
        
        
#---------------------------[Toutes le fonctions]----------------------------------------        

def ResetCanva():  #Fonction qui va reset le canva dans la fenêtre d'envoi
    f = Figure(figsize=(5,2), dpi=80)
    a = f.add_subplot(111)
            
    canva = FigureCanvasTkAgg(f, master=root)
    canva.draw()
    canva.get_tk_widget().grid(row=1, column = 0, columnspan=2, rowspan=1, stick=tk.S, pady=20)
    
def openNewWindow():  #Ouverture d'un onglet (l'historique)
     
    # Toplevel object which will
    # be treated as a new window
    History = Toplevel(root)
    History.columnconfigure(0, weight=1)
    History.columnconfigure(1, weight=1)
    # sets the title of the
    # Toplevel widget
    History.title("History")
    History.attributes('-topmost', 1) 
 
    # sets the geometry of toplevel
    History.geometry("250x300")
 
    # A Label widget to show in toplevel    
    if len(Message.Mess) == 0:
        Label = ttk.Label(History, text ="Pas de message transmis.").pack()
    else:
        Label = ttk.Label(History, text ="Liste de tous les messages transmis : (" + (str(len(Message.Mess))) + ")").grid(row=0, column=0, stick=tk.NW)
        for i in range(len(Message.Mess)):
            Label = ttk.Label(History, text =str(i+1)+ ". " + Message.Mess[i].message).grid(row=i+1, column=0, stick=tk.NW, pady=3)
            Bouton = ttk.Button(History, text="Playfile : " + Message.Mess[i].message, command= lambda : LireWAV(Message.Mess[i].message+Message.Mess[i].modulation+".wav", Fe=44100)).grid(row=i+1, column=0, stick=tk.NE)
    
def LireWAV(nom, Fe):            #Fonction que permet de lire un wav
    fs, son = wavfile.read(nom)
    print("Playing : " + str(nom))
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                            channels=len(son.shape),
                            rate=fs,
                            output=True)

    stream.write(son.tobytes())
    stream.stop_stream()
    stream.close()
    p.terminate()
    
def DestroyFiles(source=os.listdir(os.getcwd())):  #Fonction qui va détruire tous les .csv et les .wav dans le dossier
    for file in source:                            #où se trouve le script
        if file.endswith(".wav") or file.endswith(".csv"):
            os.remove(file)
        else:
            pass
        
def OpenFiles():             #Fonction qui s'occupe de l'importation du fichier .csv à démoduler
    source = os.getcwd()
    NomDuFichier = filedialog.askopenfilename(initialdir = source,
                                              title = "Select a File",
                                              filetypes = ((("CSV Files","*.csv"),)))

    label_file_explorer.configure(text="→" + NomDuFichier)
    fichier.set(NomDuFichier)
        
def demood():      #Fonction qui s'occupe d'initialiser le démodulage
    print(fichier.get())
    if fichier.get() == "None":
        print("non implémenté")
    else:
        x = Demodulation(fichier.get())

print("Lancement du programme...")

root = tk.Tk()      #Création d'une nouvelle fenêtre root
root.attributes('-topmost', 1) #On définit le titre de la fenêtre
root.title('Cesi Ton Signal -- Envoie') #On définit le titre de la fenêtre
root.geometry('500x400')  #taille de la fenêtre
root.iconbitmap('./Spy.ico') #icone


root.columnconfigure(0, weight=1)  #Définition de l'agencement de la fenêtre (en grille)
root.columnconfigure(1, weight=1)
root.rowconfigure(0, weight=1)
root.rowconfigure(1, weight=10)
root.rowconfigure(2, weight=1)

#------------------[Tous les Widgets]--------------------------------------------

message = tk.StringVar()
message2 = tk.StringVar()
Transmissions = tk.StringVar()
Transmissions.set("Nombre de transmition : {}".format(Message.NombreMessage))



Label = ttk.Label(root, text="Entrer le texte à transférer :").grid(row=0, column=0, sticky=tk.W, padx=5, pady=5)
textbox = ttk.Entry(root, textvariable=message).grid(row=1, padx=5, stick=tk.NW, ipadx=60)
Error = ttk.Label(root, text="Erreur ! Il faut saisir un message !")
ErrorModulation = ttk.Label(root, text="Erreur ! Veuillez sélectionner correctement la modulation !")
ErrorCheckSum = ttk.Label(root, text="Erreur ! Veuillez sélectionner correctement les CheckSum !")
msg_transmis = ttk.Label(root, textvariable=message2)
button_valide = ttk.Button(root, text="Valider", command= lambda : ButtonClicked("valider")).grid(row=1, column=1, stick=tk.NW, padx = 5)
button_predef = ttk.Button(root, text="Texte prédéfini [SOS]", command= lambda : ButtonClicked("prédéfini")).grid(row=1, column=1, stick=tk.NW, padx = 85, ipadx=45) #Send SOS
Historique = ttk.Button(root, text="Historique des transmitions", command=openNewWindow).grid(row=3, column=1,columnspan=1, stick=tk.SE, ipadx=16)
Nb_transmission = ttk.Label(root, textvariable=Transmissions).grid(row=3, column=0, stick=tk.W)



#------------------[CheckButton → Type de modulation]----------------------------

checkbox_ASK = tk.StringVar()
checkbox_ASK.set(False)
checkbox_FSK = tk.StringVar()
checkbox_FSK.set(False)

y_component = 90

ModulationLabel = ttk.Label(root, text="Type de Modulation : ").grid(row = 1, column =0, stick=tk.NW, pady=60, padx = 10)
ASKCheckBox = ttk.Checkbutton(root, text="ASK", variable=checkbox_ASK).grid(row = 1, column =0, pady=y_component, stick=tk.NW, padx = 70)
FSKCheckBox = ttk.Checkbutton(root, text="FSK", variable=checkbox_FSK).grid(row = 1, column =0, pady=y_component, stick=tk.NW, padx = 20)

#------------------[CheckButton → Checksum]----------------------------

checkbox_LRC = tk.StringVar()
checkbox_LRC.set(False)
checkbox_VRC = tk.StringVar()
checkbox_VRC.set(False)
checkbox_CRC = tk.StringVar()
checkbox_CRC.set(False)

CheckSumLabel = ttk.Label(root, text="Type de CheckSum : ").grid(row = 1, column =1, stick=tk.NW, pady=60, padx = 10)
VRC = ttk.Checkbutton(root, text="VRC", variable=checkbox_VRC).grid(row = 1, column =1, pady=y_component, stick=tk.NW, padx=30)
LRC = ttk.Checkbutton(root, text="LRC", variable=checkbox_LRC).grid(row = 1, column =1, pady=y_component, stick=tk.NW, padx=80)
CRC = ttk.Checkbutton(root, text="CRC", variable=checkbox_CRC).grid(row = 1, column =1, pady=y_component, stick=tk.NW, padx=130, ipadx=35)

ResetCanva()
DestroyFiles()

#----------------------------[Fenetre Reception]-----------------------------------------

Recep= tk.Tk()
Recep.attributes('-topmost', 1) 
Recep.title('Cesi Ton Signal -- Réception')
Recep.geometry('500x400')
Recep.iconbitmap('./Spy.ico')

fichier = tk.StringVar()
fichier.set("None")

selection = ttk.Label(Recep, text = "Démodulation -- Sélectionner un CSV :").grid(column=0, row=0, stick=tk.NW, padx=5)
Bouton = ttk.Button(Recep, text="Sélectionner un fichier", command=OpenFiles).grid(column=0, row=1, stick=tk.NW, pady=5, padx=10)
label_file_explorer = ttk.Label(Recep, text = "")
separatorLabel = ttk.Label(Recep, text="              -----------------------------------------------------------------------------------").grid(column=0, row=2, stick=tk.NW, columnspan=1)
Demodulation_button = ttk.Button(Recep, text="Démoduler", command=demood).grid(column=0, row=2, pady=25, padx=200, stick=tk.NW)
label_file_explorer.grid(column=0, row=1, stick=tk.W, pady=5, padx=150, columnspan=1)
ASK_decode = ttk.Label(Recep, text="")
Error_ASK = ttk.Label(Recep, text="")
ASK_decode.grid(column=0, row=3, padx=10, stick=tk.W, columnspan=1)
Error_ASK.grid(column=0, row=4, padx=10, stick=tk.W, columnspan=1)
FSK_decode = ttk.Label(Recep, text="")
Error_FSK = ttk.Label(Recep, text="")
FSK_decode.grid(column=0, row=5, padx=10,stick=tk.W, columnspan=1)
Error_FSK.grid(column=0, row=6, padx=10, stick=tk.W, columnspan=1)

mainloop() #Pour laisser la fenêtre affichée (boucle)

#for i in range(len(Message.Mess)):
#    print(Message.Mess[i].message, Message.Mess[i].modulation, Message.Mess[i].checksum)

### 